# Topic Modeling of Twitter Statuses with NMF TODO

In [4]:
from gensim import corpora, models, similarities
from pymongo import MongoClient
import time
import os
import random
import numpy as np
import pyLDAvis.gensim
import csv
from collections import defaultdict

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../')

from src.streaming import spark_functions

preprocess = spark_functions.preprocessor()
tokenize = spark_functions.tokenizer(remove_stopwords=True)

## Creating the Corpus and Dictionary

In [5]:
# Get the tweets to be used
with open('./../../data/interim/sanders_hydrated_en.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Preprocess with the same preprocessing and tokenization function as usual
    tweets = [" ".join(tokenize(preprocess(text))) for (topic, sentiment, id, text) in iterator]

## Train the NMF model

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(tweets)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(tweets)
tf_feature_names = tf_vectorizer.get_feature_names()

In [11]:

from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 5

# Run NMF
nmf = NMF(n_components=no_topics, 
          random_state=1, 
          alpha=.1, 
          l1_ratio=.5, 
          init='nndsvd').fit(tfidf)

## Some Experiments

In [12]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 5
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
cream ice sandwich android dhilipsiva
Topic 1:
nexus galaxy samsung android ics
Topic 2:
iphone million weekend battery sales
Topic 3:
ballmer steve windows android yahoo
Topic 4:
siri video network amazing gets


In [7]:
# Pie chart of topics
topic_frequency = defaultdict(int)
for tweet in tweets:
    topics = lda[dictionary.doc2bow(tweet[0])]
    for topic in topics:
        #print(topic)
        topic_frequency[topic[0]] += topic[1]
        

import matplotlib.pyplot as plt
labels,values = [],[]
for key in topic_frequency:
    values.append(topic_frequency[key])
    labels.append(", ".join(dictionary[term_id] for (term_id,_) in lda.get_topic_terms(topicid=key, topn=5)))
plt.pie(values, labels=labels, shadow=False, autopct='%1.1f%%',)
plt.savefig('topic_distribution.pdf', format="pdf")
plt.show()

NameError: name 'lda' is not defined

In [55]:
def print_topics(text):
    doc_bow = [dictionary.doc2bow(text) for text in [tokenize(preprocess(text))]]
    topics = lda[doc_bow][0]

    print(text)
    for topic in topics:  # Maximum 3 Topics per tweet
        terms = lda.get_topic_terms(topicid=topic[0], topn=5) # Maximum 5 Terms per tweet
        print("Probability %d%%: %s" % (topic[1]*100, ", ".join(dictionary[term[0]] for term in terms)))
        
print_topics("lol i hate the stupid #iphone")
print_topics("android is pretty cool I guess, with the new ice cream sandwich update")

lol i hate the stupid #iphone
Probability 99%: ballmer, steve, yahoo, aqui, free
android is pretty cool I guess, with the new ice cream sandwich update
Probability 57%: android, nexus, samsung, galaxy, cream
Probability 42%: ballmer, steve, yahoo, aqui, free


In [28]:
print(len(doc_lda))
print(len(corpus))
print(len(dictionary))
print(len(followers_data))
followers_data[0]

1
4347
3957
7


,Freq,cluster,topics,x,y
topic,,,,,
4,29.385533,1,1,-0.113680,0.183263
0,21.947690,1,2,-0.120678,0.046538
1,19.418286,1,3,-0.042073,-0.211118
2,14.723970,1,4,0.354404,0.037260
3,14.524521,1,5,-0.077973,-0.055943


In [ ]:
# How to access which topic the nth is?